In [86]:
! mkdir joy

In [87]:
cd joy/

/content/joy


## Downloading All Joy dataset

In [88]:
!wget https://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/joy-ratings-0to1.train.txt
!wget https://saifmohammad.com/WebDocs/EmoInt%20Dev%20Data/joy-ratings-0to1.dev.target.txt
!wget https://saifmohammad.com/WebDocs/EmoInt%20Dev%20Data%20With%20Gold/joy-ratings-0to1.dev.gold.txt
!wget https://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/joy-ratings-0to1.test.target.txt
!wget https://saifmohammad.com/WebDocs/EmoInt%20Test%20Gold%20Data/joy-ratings-0to1.test.gold.txt

--2020-12-04 16:55:45--  https://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/joy-ratings-0to1.train.txt
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92266 (90K) [text/plain]
Saving to: ‘joy-ratings-0to1.train.txt’

joy-ratings-0to1.tr 100%[===================>]  90.10K   228KB/s    in 0.4s    

2020-12-04 16:55:47 (228 KB/s) - ‘joy-ratings-0to1.train.txt’ saved [92266/92266]

--2020-12-04 16:55:47--  https://saifmohammad.com/WebDocs/EmoInt%20Dev%20Data/joy-ratings-0to1.dev.target.txt
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8665 (8.5K) [text/plain]
Saving to: ‘joy-ratings-0to1.dev.target.txt’

joy-ratings-0to1.de 100%[===================>]   8.46K  --.-KB/s    in 0s   

In [89]:
cd ..

/content


In [90]:
!pip install emot
!pip install wordsegment

## Importing All libraries

In [91]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.base import TransformerMixin

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

import re
import nltk
from nltk.util import ngrams  
import collections
from nltk.corpus import stopwords
from nltk import word_tokenize
from wordsegment import segment, load
from nltk.tokenize import TweetTokenizer
nltk.download("stopwords")
load()
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

from keras.models import Sequential
from keras.layers import Dense

from scipy.stats import spearmanr
from scipy.stats import pearsonr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:

train = pd. read_csv("./joy/joy-ratings-0to1.train.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
val = pd. read_csv("./joy/joy-ratings-0to1.dev.target.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
val_gold = pd. read_csv("./joy/joy-ratings-0to1.dev.gold.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
test = pd. read_csv("./joy/joy-ratings-0to1.test.target.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])
test_gold = pd. read_csv("./joy/joy-ratings-0to1.test.gold.txt", sep="\t", names=['id', 'tweet', 'emotion', 'score'])


## Pre-Processing Script

In [93]:
tknzr = TweetTokenizer(reduce_len=True, preserve_case=False, strip_handles=False)
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

def text_preprocess(text):
    FLAGS = re.MULTILINE | re.DOTALL
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = re_sub(r"#\S+", lambda hashtag: " ".join(segment(hashtag.group()[1:]))) # segment hastags

    tokens = tknzr.tokenize(text.lower())
    return " ".join(tokens)

## Simple Evaluation Script based on [This Evaluation](https://github.com/felipebravom/EmoInt)

In [94]:
def metrics(y_pred, y, title):
    p1 = pearsonr(y_pred, y)[0]
    s1 = spearmanr(y_pred, y)[0]
    ind = np.where(y >= 0.5)
    ydt = np.take(y_pred, ind).reshape(-1)
    ydpt = np.take(y.to_numpy(), ind).reshape(-1)
    p2 = pearsonr(ydt, ydpt)[0]
    s2 = spearmanr(ydt, ydpt)[0]
    print("********"+ title + "***********")
    res = {}
    res['Pearsonr'] = p1
    res['Spearmanr'] = s1
    res['Pearsonr >= 0.5'] = p2
    res['Spearmanr >= 0.5'] = s2
    result = pd.DataFrame(res.items(), columns=['metrics', 'Value'], index=None)
    return result

## Function to run ML Models

In [95]:
def ml_model(X_train, X_test, y_train, y_test, val_X, val_y):
    prediction_accuracy = {}
    
    # Random Forest 
    _RandomForestRegressor = RandomForestRegressor(n_estimators = 1000, oob_score=True)
    _RandomForestRegressor.fit(X_train, y_train)
    _RandomForestRegressor_prediction = _RandomForestRegressor.predict(X_test)
    val_RandomForestRegressor_prediction = _RandomForestRegressor.predict(val_X)
    
    print("*"*24,"Random Forest ","*"*24)
    print(metrics(_RandomForestRegressor_prediction, y_test, "Train-set "))
    print(metrics(val_RandomForestRegressor_prediction, val_y, "Test-set "))
    print('\n ')
    
    # Support Vector Machine
    _SVC = SVR( kernel = 'rbf')
    _SVC.fit(X_train, y_train)
    _SVC_prediction = _SVC.predict(X_test)
    val_SVC_prediction = _SVC.predict(val_X)
    
    print("*"*24, "Support Vector Machine", "*"*24)
    print(metrics(_SVC_prediction, y_test, "Train-set "))
    print(metrics(val_SVC_prediction, val_y, "Test-set "))
    print('\n ')
    
    #accuracy DataFram
    prediction_accuracy_df = pd.DataFrame(prediction_accuracy.items(), columns=['Model', 'Accuracy'], index=None)
    return prediction_accuracy_df

## Approach one - Simple Stopword


In [96]:
def simple_stopwatch_approach(X, y, val_X, val_y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    # Creating the vectorizer
    tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=20, stop_words= STOPWORDS )
    X_train = tfidf.fit_transform(X_train).toarray()
    X_test = tfidf.transform(X_test).toarray()
    val_X = tfidf.transform(val_X).toarray()
    return X_train, X_test, y_train, y_test, val_X, val_y

## Approach one - Custom vocabulary

In [97]:
nltk.download('punkt')
# Taking Title, body and comments to create custom vocablary
top1 = 3000
top2 = 3000
custom_vocab = []

text_concate = pd.concat([train, val, test], ignore_index=True)

all_text = ' '.join(str(word) for word in text_concate.tweet) 
tokens = word_tokenize(all_text)

unigrams = ngrams(tokens, 1)
unigrams = collections.Counter(unigrams)
unigrams = unigrams.most_common(top1)
for i in range(top1):
    custom_vocab.append(unigrams[i][0][0])

bigrams = ngrams(tokens, 2)
bigrams = collections.Counter(bigrams)
bigrams = bigrams.most_common(top2)
for i in range(top2):
    custom_vocab.append(bigrams[i][0][0]+' '+bigrams[i][0][1])

custom_vocab = list(set(custom_vocab))
custom_vocab = [word for word in custom_vocab if word not in STOPWORDS]

# Creating TF-IDF Vectorizer using custom custom_vocab
tfidf = TfidfVectorizer(vocabulary = custom_vocab)

    
def custom_approach(X, y, val_X, val_y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

    X_train = tfidf.fit_transform(X_train).toarray()
    X_test = tfidf.transform(X_test).toarray()
    val_X = tfidf.transform(val_X).toarray()
    return X_train, X_test, y_train, y_test, val_X, val_y

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Downloading Twitter Glove Embedding

In [39]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-12-04 15:13:51--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-12-04 15:13:51--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-12-04 15:13:52--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [40]:
!unzip './glove.twitter.27B.zip'

Archive:  ./glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


## Class to generate Embedding [Ref](https://www.kaggle.com/nhrade/text-classification-using-word-embeddings)

In [98]:
class MeanEmbeddingTransformer(TransformerMixin):
    def __init__(self):
        self._vocab, self._E = self._load_words()

    def _load_words(self):
        E = {}
        vocab = []
        with open('./glove.twitter.27B.100d.txt', 'r', encoding="utf8") as file:
            for i, line in enumerate(file):
                l = line.split(' ')
                if l[0].isalpha():
                    v = [float(i) for i in l[1:]]
                    E[l[0]] = np.array(v)
                    vocab.append(l[0])
        return np.array(vocab), E            

    def _get_word(self, v):
        for i, emb in enumerate(self._E):
            if np.array_equal(emb, v):
                return self._vocab[i]
        return None
    
    def _doc_mean(self, doc):
        return np.mean(np.array([self._E[w.lower().strip()] for w in doc if w.lower().strip() in self._E]), axis=0)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([self._doc_mean(doc) for doc in X])
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

## Approach 3 - Twitter Glove Embedding

In [99]:
def tokenize_and_transform(X, sample_size):
    tok_es1 = [word_tokenize(doc) for doc in X[:sample_size]]
    met = MeanEmbeddingTransformer()
    X_transform =met.fit_transform(tok_es1)
    return X_transform

def glove_approach(X, y, val_X, val_y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    X_train = tokenize_and_transform(X_train, len(X_train))
    X_test = tokenize_and_transform(X_test, len(X_test))
    val_X = tokenize_and_transform(val_X, len(val_X))
    return X_train, X_test, y_train, y_test, val_X, val_y

## Function to run all Approaches for ML Algo

In [100]:
def train_ml_algo(train, val, test, prePro=False):
  train_val = pd.concat([train, val], ignore_index=True)
  if prePro:
    train['tweet'] = train['tweet'].apply(convert_emojis)
    val_gold['tweet'] = val_gold['tweet'].apply(convert_emojis)
    test_gold['tweet'] = test_gold['tweet'].apply(convert_emojis)
    train_val['tweet'] = train_val['tweet'].apply(convert_emojis)

    train['tweet'] = train['tweet'].apply(text_preprocess)
    val_gold['tweet'] = val_gold['tweet'].apply(text_preprocess)
    test_gold['tweet'] = test_gold['tweet'].apply(text_preprocess)
    train_val['tweet'] = train_val['tweet'].apply(text_preprocess)
  
  if prePro:
    print("*"*24, "After Pre-Processing", "*"*24, "\n")
  else:
    print("*"*24, "Before Pre-Processing", "*"*24, "\n")

  
  # Approach 1
  print("*"*24, "Approach 1 Simple Stopwatch", "*"*24)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = simple_stopwatch_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = simple_stopwatch_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)


  # Approach 2
  print("*"*24, "Approach 2 Custome vocabulary", "*"*24)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = custom_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = custom_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)

  # Approach 3
  print("="*24, "Approach 3 Twitter Glove Embedding", "="*21)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = glove_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = glove_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = ml_model(X_train, X_test, y_train, y_test, val_X, val_y)


## Without Pre-processing

In [101]:
train_ml_algo(train, val_gold, test_gold, prePro=False)

************************ Before Pre-Processing ************************ 

************************ Approach 1 Simple Stopwatch ************************
************************ Train  => Validation ************************
************************ Random Forest  ************************
********Train-set ***********
            metrics     Value
0          Pearsonr  0.497843
1         Spearmanr  0.465788
2   Pearsonr >= 0.5  0.409247
3  Spearmanr >= 0.5  0.388459
********Test-set ***********
            metrics     Value
0          Pearsonr  0.297187
1         Spearmanr  0.307241
2   Pearsonr >= 0.5  0.263739
3  Spearmanr >= 0.5  0.301361

 
************************ Support Vector Machine ************************
********Train-set ***********
            metrics     Value
0          Pearsonr  0.473967
1         Spearmanr  0.437280
2   Pearsonr >= 0.5  0.385501
3  Spearmanr >= 0.5  0.404218
********Test-set ***********
            metrics     Value
0          Pearsonr  0.256570
1       

## With Pre-processing

In [102]:
train_ml_algo(train, val_gold, test_gold, prePro=True)

************************ After Pre-Processing ************************ 

************************ Approach 1 Simple Stopwatch ************************
************************ Train  => Validation ************************
************************ Random Forest  ************************
********Train-set ***********
            metrics     Value
0          Pearsonr  0.477274
1         Spearmanr  0.433205
2   Pearsonr >= 0.5  0.320726
3  Spearmanr >= 0.5  0.311084
********Test-set ***********
            metrics     Value
0          Pearsonr  0.278302
1         Spearmanr  0.321975
2   Pearsonr >= 0.5  0.203804
3  Spearmanr >= 0.5  0.201400

 
************************ Support Vector Machine ************************
********Train-set ***********
            metrics     Value
0          Pearsonr  0.494236
1         Spearmanr  0.471457
2   Pearsonr >= 0.5  0.348926
3  Spearmanr >= 0.5  0.364503
********Test-set ***********
            metrics     Value
0          Pearsonr  0.218412
1        

## Funtion to define Simple MLP

In [103]:
def NN_model(X_train, X_test, y_train, y_test, val_X, val_y):

  model = Sequential()
  model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(256, kernel_initializer='normal',activation='relu'))
  model.add(Dense(1, kernel_initializer='normal',activation='linear'))
  model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

  model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.2)
  pre = model.predict(X_test)
  p = []
  for i in pre:
    p.append(i[0])
  print(metrics(p, y_test, "Train-set "))
  pre_val = model.predict(val_X)
  p_val = []
  for i in pre_val:
    p_val.append(i[0])
  print(metrics(p_val, val_y, "Test-set "))
  
  return None


## Function to run all Approaches for MLP

In [104]:
def train_NN(train, val, test, prePro=False):
  train_val = pd.concat([train, val], ignore_index=True)
  if prePro:
    train['tweet'] = train['tweet'].apply(convert_emojis)
    val_gold['tweet'] = val_gold['tweet'].apply(convert_emojis)
    test_gold['tweet'] = test_gold['tweet'].apply(convert_emojis)
    train_val['tweet'] = train_val['tweet'].apply(convert_emojis)

    train['tweet'] = train['tweet'].apply(text_preprocess)
    val_gold['tweet'] = val_gold['tweet'].apply(text_preprocess)
    test_gold['tweet'] = test_gold['tweet'].apply(text_preprocess)
    train_val['tweet'] = train_val['tweet'].apply(text_preprocess)
  
  if prePro:
    print("*"*24, "After Pre-Processing", "*"*24, "\n")
  else:
    print("*"*24, "Before Pre-Processing", "*"*24, "\n")

  
  # Approach 1
  print("*"*24, "Approach 1 Simple Stopwatch", "*"*24)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = simple_stopwatch_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = simple_stopwatch_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)


  # Approach 2
  print("*"*24, "Approach 2 Custome vocabulary", "*"*24)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = custom_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = custom_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)

  # Approach 3
  print("="*24, "Approach 3 Twitter Glove Embedding", "="*21)
  # Train
  print("*"*24, "Train  => Validation", "*"*24)
  X_train, X_test, y_train, y_test,val_X, val_y = glove_approach(train.tweet ,train.score , val_gold.tweet , val_gold.score )
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)
  # Test
  print("*"*24, "Train + Validation => Test", "*"*24, "\n")
  X_train, X_test, y_train, y_test,val_X, val_y = glove_approach(train_val.tweet ,train_val.score , test_gold.tweet , test_gold.score)
  p_df = NN_model(X_train, X_test, y_train, y_test, val_X, val_y)


In [105]:
train_NN(train, val_gold, test_gold, prePro=False)

************************ Before Pre-Processing ************************ 

************************ Approach 1 Simple Stopwatch ************************
************************ Train  => Validation ************************
Epoch 1/100
17/17 [==============================] - 1s 36ms/step - loss: 0.2729 - mean_absolute_error: 0.2729 - val_loss: 0.1786 - val_mean_absolute_error: 0.1786
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 0.1684 - mean_absolute_error: 0.1684 - val_loss: 0.1578 - val_mean_absolute_error: 0.1578
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1549 - mean_absolute_error: 0.1549 - val_loss: 0.1678 - val_mean_absolute_error: 0.1678
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1627 - mean_absolute_error: 0.1627 - val_loss: 0.1532 - val_mean_absolute_error: 0.1532
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1479 - mean_absolute_error: 0.1479 - val_loss: 0.

In [106]:
train_NN(train, val_gold, test_gold, prePro=True)

************************ After Pre-Processing ************************ 

************************ Approach 1 Simple Stopwatch ************************
************************ Train  => Validation ************************
Epoch 1/100
17/17 [==============================] - 0s 14ms/step - loss: 0.2970 - mean_absolute_error: 0.2970 - val_loss: 0.1989 - val_mean_absolute_error: 0.1989
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1653 - mean_absolute_error: 0.1653 - val_loss: 0.1694 - val_mean_absolute_error: 0.1694
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1575 - mean_absolute_error: 0.1575 - val_loss: 0.1579 - val_mean_absolute_error: 0.1579
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1649 - mean_absolute_error: 0.1649 - val_loss: 0.1640 - val_mean_absolute_error: 0.1640
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 0.1641 - mean_absolute_error: 0.1641 - val_loss: 0.1